# Urban Tree Canopy - Part 2 (UTC Processing)

## After completing Part 1 (LiDAR Processing or Raster Classification) AND the output has been QA/QC'd, this script will process the polygons into UTC and C2C shapefiles that are ready to be uploaded.

## BEFORE YOU BEGIN
1.) Ensure the output for Part 1 has been uploaded to a new GDB in your folders
2.) Import the outputs from Part 1 into your map and ensure its active (i.e. you can still see the layers on the map while in this notebook)
3.) Import the CHHD, City_Boundary, and ETJ shapefiles into the same map. The ETJ and City Boundary layers will be essentially the same, with only the polygon being different.
4.) Change the input in the first cell to reflect where the newly created GDB from Part 1 is, and where you want the final outputs to be saved.


In [5]:
import os
#CHANGE THESE
final_gdb = r'D:\ArcGIS_Projects\UTC\UTC\UTC_Final.gdb'
arcpy.env.workspace = r'D:\ArcGIS_Projects\UTC\UTC\Polygons_from_ETJ_Within_Ellis_hill_2022.gdb'
#DON'T CHANGE THESE
feature_classes = arcpy.ListFeatureClasses()
cities = []
for fc in feature_classes:
    cities.append(fc)
print(cities)


['Corsicana', 'Ennis', 'Hillsboro']


In [9]:
arcpy.env.overwriteOutput = True
arcpy.env.workspace = arcpy.env.scratchGDB
for city in cities:
    print(f'Starting f'{city}' analysis.)
    arcpy.management.Dissolve(f'{city}', f'{city}')
    arcpy.management.SelectLayerByAttribute('ETJ', 'NEW_SELECTION', f"NAME10 = '{city}'")
    arcpy.analysis.Clip('CHHD', 'ETJ', f'CHHD_{city}')
    arcpy.analysis.Clip(f'{city}', 'ETJ', f'{city}_ETJ_TEMP')
    arcpy.management.SelectLayerByAttribute('City_Boundary', 'NEW_SELECTION', f"NAME10 = '{city}'")
    arcpy.analysis.Clip(f'{city}', 'City_Boundary', f'{city}_City')
    arcpy.management.SelectLayerByAttribute('City_Boundary', "CLEAR_SELECTION")
    arcpy.management.SelectLayerByAttribute('ETJ', "CLEAR_SELECTION")
    arcpy.analysis.Erase(f'{city}_ETJ_TEMP', f'{city}_City', f'{city}_ETJ')
    arcpy.management.AddFields(f'{city}_City', [['Name', 'TEXT'], 
                                                 ['CL_ET', 'SHORT'],
                                                ['Area', 'FLOAT'],
                                                ['PlaceFIPS', 'LONG']])
    
    arcpy.management.AddFields(f'{city}_ETJ', [['Name', 'TEXT'], 
                                                 ['CL_ET', 'SHORT'],
                                                ['Area', 'FLOAT'],
                                               ['PlaceFIPS', 'LONG']])
    arcpy.management.CalculateField(f'{city}_ETJ', "Name", f"'{city}'", 'PYTHON3')
    arcpy.management.CalculateField(f'{city}_ETJ', "CL_ET", -1 , "PYTHON3")
    arcpy.management.CalculateGeometryAttributes(f'{city}_ETJ', [['Area', 'AREA_GEODESIC']],"METERS","ACRES_US")
    arcpy.management.JoinField(f'{city}_ETJ', 'Name', 'ETJ','Name10', 'GRIDCODE')
    arcpy.management.CalculateField(f'{city}_ETJ', "PlaceFIPS", '!GRIDCODE!', "PYTHON3")
    arcpy.management.DeleteField(f'{city}_ETJ', 'GRIDCODE')
    arcpy.management.CalculateField(f'{city}_ETJ', "Name", f"'{city}_ETJ'", 'PYTHON3')
    
    arcpy.management.CalculateField(f'{city}_City', "Name", f"'{city}'", "PYTHON3")
    arcpy.management.CalculateField(f'{city}_City', "CL_ET", 0 , "PYTHON3")
    arcpy.management.CalculateGeometryAttributes(f'{city}_City', [['Area', 'AREA_GEODESIC']], "METERS", "ACRES_US")
    arcpy.management.JoinField(f'{city}_City', 'Name', 'ETJ','Name10', 'GRIDCODE')
    arcpy.management.CalculateField(f'{city}_City', "PlaceFIPS", '!GRIDCODE!', "PYTHON3")
    arcpy.management.DeleteField(f'{city}_City', 'GRIDCODE')  
    
    arcpy.management.Merge([f'{city}_City', f'{city}_ETJ'], f'{city}_utc')

    arcpy.management.AddField(f'{city}_utc', 'C2C', 'TEXT')
    arcpy.management.CalculateField(f'{city}_utc', 'C2C', "'At Risk'")    
    
    arcpy.analysis.Erase(f'CHHD_{city}', f'{city}_utc', f'{city}_c2c_temp')
    arcpy.management.AddFields(f'{city}_c2c_temp', [['Name', 'TEXT'], 
                                                 ['CL_ET', 'SHORT'],
                                                ['Area', 'FLOAT'],
                                               ['PlaceFIPS', 'LONG']])
    arcpy.management.CalculateField(f'{city}_c2c_temp', "Name", f"'{city}'", "PYTHON3")
    arcpy.management.CalculateField(f'{city}_c2c_temp', "CL_ET", 2 , "PYTHON3")
    arcpy.management.CalculateGeometryAttributes(f'{city}_c2c_temp', [['Area', 'AREA_GEODESIC']], "METERS", "ACRES_US")
    arcpy.management.DeleteField(f'{city}_c2c_temp', 'GRIDCODE') 
    arcpy.management.JoinField(f'{city}_c2c_temp', 'Name', 'ETJ','Name10', 'GRIDCODE')
    arcpy.management.CalculateField(f'{city}_c2c_temp', "PlaceFIPS", '!GRIDCODE!', "PYTHON3")
    arcpy.management.DeleteField(f'{city}_c2c_temp', 'GRIDCODE') 
    arcpy.management.AddField(f'{city}_utc', 'C2C', 'TEXT')
    arcpy.management.CalculateField(f'{city}_c2c_temp', 'C2C', "'To Grow'")
    
    arcpy.management.Merge([f'{city}_utc', f'{city}_c2c_temp'], f'{city}_c2c')
    
    output_utc = os.path.join(final_gdb, f'{city}_utc')
    arcpy.management.CopyFeatures(f'{city}_utc', output_utc)
    
    output_c2c = os.path.join(final_gdb, f'{city}_c2c')
    arcpy.management.CopyFeatures(f'{city}_c2c', output_c2c)
    print(f'Analysis completed for {city}')
    